# Multilabel Classification

In [1]:
import numpy as np
import pandas as pd
import lmdb
import os
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier

## Importing Data

In [2]:
data_path = 'C:/Users/Elwin/Desktop/Data Science & Analytics/Year 3/CS3244/Project/Data'
keys_path = 'C:/Users/Elwin/Desktop/Data Science & Analytics/Year 3/CS3244/Project/Data/val_keys.pkl'
name_path = 'C:/Users/Elwin/Desktop/Data Science & Analytics/Year 3/CS3244/Project/Data/classes1M.pkl'

env = lmdb.open(os.path.join(data_path, 'val_lmdb'), max_readers=1, readonly=True, lock=False, readahead=False, meminit=False)

ids = pickle.load(open(keys_path, 'rb'))
f = open(name_path, 'rb')
_ = pickle.load(f)
categories = pickle.load(f)

for i in range(20):
  print("{}: {}".format(i, categories[i]))

0: background
1: peanut butter
2: ice cream
3: chocolate chip
4: sweet potato
5: crock pot
6: pork chops
7: cream cheese
8: black bean
9: potato salad
10: chicken salad
11: sour cream
12: goat cheese
13: pasta salad
14: salad recipe
15: green beans
16: chip cookies
17: blue cheese
18: french toast
19: butternut squash


In [3]:
storage = []
classes = []
def path_for(filename):
  return os.path.join(images_path, filename[0], filename[1], filename[2], filename[3], filename)

for id in ids:
  with env.begin(write=False) as txn:
    byte_sample = txn.get(id.encode())
  sample = pickle.loads(byte_sample, encoding='latin')
  sample.pop('imgs',None)
  sample['intrs'] = np.average(sample['intrs'], axis=0)
  #sample['intrs'] = np.sum(sample['intrs'], axis=0) #I used average instead as it is more recommended on the web
  entry = np.concatenate([sample['ingrs'],sample['intrs']])
  classes.append(sample['classes'])
  storage.append(entry.tolist())

## Normalisation

In [4]:
df = pd.DataFrame(storage)
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df).tolist())
df["classes"] = classes
dummies = pd.get_dummies(df["classes"])
dummies.columns = ['class' + str(col) for col in dummies.columns]
df = pd.concat([df, dummies], axis=1) 

(51129, 1044)


## Replace the the train and test here

In [5]:
train, test = train_test_split(df, random_state=42, test_size=0.30, shuffle=True)
x_train = train.iloc[:, 0:1043]
y_train = train.iloc[:,1045:]
x_test = test.iloc[:, 0:1043]
y_test = test.iloc[:,1045:]
allcategories = list(y_test.columns.values)

## OnevsRestClassifier

In [6]:
multilabels = pd.DataFrame()
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in allcategories:
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    multilabels[category] = prediction #This is the multilabel dataset
    #print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    #print("\n")

**Processing class1 comments...**
[1 1 1 ... 1 1 0]
